# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [11]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re 
import nltk
nltk.download(['punkt', 'wordnet','stopwords','words'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [12]:
# load data from database
engine = create_engine('sqlite:///test_DB.db')
df = pd.read_sql_table('test_table',engine)
X = df['message']
y = df.iloc[:, 3:]
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [13]:

def tokenize(text):
    tokens = re.sub(r'[^a-zA-Z0-9]',' ', text)
    
    tokens = word_tokenize(tokens)
    
    # Remove stop words
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    #Initialize Stemmer and Lemmatizer
    ps = PorterStemmer() 
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        # Stemming to reduce to a common root
        clean_tok = ps.stem(tok).lower().strip()
        
        #Lemmatizing to reduce to the actual root
        clean_tok = lemmatizer.lemmatize(clean_tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



In [14]:
class SentenceLengthExtractor(BaseEstimator, TransformerMixin):

    def sen_len(self, text):
        tokens = nltk.word_tokenize(text)
        return len(tokens)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.sen_len)
        return pd.DataFrame(X_tagged)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
    pipeline = Pipeline([
                ('c_vect', CountVectorizer(tokenizer=tokenize)),
                ('c_tfidf_trans', TfidfTransformer()),
                ('c_clf_adabost', MultiOutputClassifier(AdaBoostClassifier(DecisionTreeClassifier(max_depth = 1))))

        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('c_vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        str...er='best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [18]:
y_pred = pipeline.predict(X_test)

In [19]:
pipeline.score(X_test, y_test)

0.22093378089716204

In [20]:
accuracy = ((y_test == y_pred).mean()).values.mean()
print(accuracy)

0.946826365578


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
y_pred = pipeline.predict(X_test)

In [22]:
import pprint as pp
pp.pprint(sorted(pipeline.get_params().keys()))

['c_clf_adabost',
 'c_clf_adabost__estimator',
 'c_clf_adabost__estimator__algorithm',
 'c_clf_adabost__estimator__base_estimator',
 'c_clf_adabost__estimator__base_estimator__class_weight',
 'c_clf_adabost__estimator__base_estimator__criterion',
 'c_clf_adabost__estimator__base_estimator__max_depth',
 'c_clf_adabost__estimator__base_estimator__max_features',
 'c_clf_adabost__estimator__base_estimator__max_leaf_nodes',
 'c_clf_adabost__estimator__base_estimator__min_impurity_decrease',
 'c_clf_adabost__estimator__base_estimator__min_impurity_split',
 'c_clf_adabost__estimator__base_estimator__min_samples_leaf',
 'c_clf_adabost__estimator__base_estimator__min_samples_split',
 'c_clf_adabost__estimator__base_estimator__min_weight_fraction_leaf',
 'c_clf_adabost__estimator__base_estimator__presort',
 'c_clf_adabost__estimator__base_estimator__random_state',
 'c_clf_adabost__estimator__base_estimator__splitter',
 'c_clf_adabost__estimator__learning_rate',
 'c_clf_adabost__estimator__n_esti

In [23]:
for label, pred, col in zip(y_test.values.transpose(), y_pred.transpose(), 
                                category_names):
        print(col)
        print(classification_report(label, pred))
    

related
             precision    recall  f1-score   support

          0       0.59      0.08      0.14      1524
          1       0.77      0.98      0.86      4975
          2       0.43      0.11      0.17        55

avg / total       0.73      0.77      0.69      6554

request
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      5398
          1       0.74      0.53      0.62      1156

avg / total       0.88      0.88      0.88      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.76      0.85      0.80      3842
          1       0.75      0.62      0.67      2712

avg / total       0.75      0.75      0.75      6554

medical_help
             precision    recall  f1-sco

### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
parameters = {
    'c_clf_adabost__estimator__base_estimator__max_depth' : [1,2,3]
}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv = 3, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
cv.best_params_

In [ ]:
print(cv.best_score_)
print(cv.scorer_)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.